# 🚄 TARDIS - Analyse Avancée des Retards de Trains
Ce notebook réalise un nettoyage complet des données, les valide avec la liste officielle des gares, puis produit des visualisations avancées pour mieux comprendre les retards à travers la France.

## 📦 Imports & Configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rapidfuzz import distance

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
%matplotlib inline

## 📂 Chargement des Données

In [ ]:
# Charger le dataset principal
try:
    df = pd.read_csv("dataset.csv", sep=";", on_bad_lines="skip")
    print("✅ Données chargées :", df.shape)
except Exception as e:
    print(f"Erreur lors du chargement du dataset principal : {e}")
    df = pd.DataFrame()

# Charger la liste des gares
try:
    gares = pd.read_csv("liste-des-gares.csv", sep=";")
    gares["LIBELLE"] = gares["LIBELLE"].str.lower().str.strip().str.replace("-", " ")
except Exception as e:
    print(f"Erreur lors du chargement de la liste des gares : {e}")
    gares = pd.DataFrame()

# Charger la liste des villes mondiales
try:
    cities = pd.read_csv("worldcities.csv")
    # On peut préparer les données des villes si nécessaire
    cities["city_lower"] = cities["city_ascii"].str.lower().str.strip()
    print("✅ Données des villes mondiales chargées :", cities.shape)
except Exception as e:
    print(f"Erreur lors du chargement des villes mondiales : {e}")
    cities = pd.DataFrame()

# Vérification
if df.empty or gares.empty or cities.empty:
    print(
        "❌ Les fichiers nécessaires n'ont pas été chargés correctement. Arrêt du programme."
    )

## 🧹 Nettoyage des Données

In [ ]:
# Sélection des colonnes d'intérêt
df = df[
    [
        "Date",
        "Departure station",
        "Arrival station",
        "Average delay of all trains at departure",
        "Average delay of all trains at arrival",
        "Number of scheduled trains",
        "Number of cancelled trains",
        "Arrival delay comments",
        "Number of trains delayed at departure",
        "Number of trains delayed at arrival",
        "Departure delay comments",
        "Number of trains delayed > 15min",
        "Number of trains delayed > 30min",
        "Number of trains delayed > 60min",
        "Pct delay due to external causes",
        "Pct delay due to infrastructure",
        "Pct delay due to traffic management",
        "Pct delay due to rolling stock",
        "Pct delay due to station management and equipment reuse",
        "Pct delay due to passenger handling (crowding, disabled persons, connections)",
    ]
]
# Renommage des colonnes
df.columns = [
    "date",
    "departure_station",
    "arrival_station",
    "avg_dep_delay",
    "avg_arr_delay",
    "scheduled_trains",
    "cancelled_trains",
    "arrival_delay_comments",
    "trains_delayed_dep",
    "trains_delayed_arr",
    "departure_delay_comments",
    "trains_delayed_15min",
    "trains_delayed_30min",
    "trains_delayed_60min",
    "pct_delay_external",
    "pct_delay_infrastructure",
    "pct_delay_traffic_mgmt",
    "pct_delay_rolling_stock",
    "pct_delay_station_mgmt",
    "pct_delay_passenger",
]
# Nettoyage des colonnes texte
df["departure_station"] = df["departure_station"].str.lower().str.strip()
df["arrival_station"] = df["arrival_station"].str.lower().str.strip()
# Conversion en types numériques
numeric_cols = [
    "avg_dep_delay",
    "avg_arr_delay",
    "scheduled_trains",
    "cancelled_trains",
    "trains_delayed_dep",
    "trains_delayed_arr",
    "trains_delayed_15min",
    "trains_delayed_30min",
    "trains_delayed_60min",
    "pct_delay_external",
    "pct_delay_infrastructure",
    "pct_delay_traffic_mgmt",
    "pct_delay_rolling_stock",
    "pct_delay_station_mgmt",
    "pct_delay_passenger",
]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
# Conversion des dates
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Correction des valeurs négatives : on met à 0 si < 0, puis on limite à scheduled_trains
for col in ["cancelled_trains", "trains_delayed_dep", "trains_delayed_arr"]:
    df[col] = df[col].clip(lower=0)
    df[col] = df[[col, "scheduled_trains"]].min(axis=1)

# Filtrage des années valides (2018 à 2024 inclus)
annee_min, annee_max = 2018, 2024
before_year_filter = df.shape[0]
df = df[df["date"].dt.year.between(annee_min, annee_max)]
after_year_filter = df.shape[0]

print(
    f"✅ Lignes conservées entre {annee_min} et {annee_max} : {after_year_filter} (sur {before_year_filter})"
)
# Suppression des lignes incomplètes pour les colonnes essentielles
df.dropna(
    subset=["date", "departure_station", "avg_dep_delay", "scheduled_trains"],
    inplace=True,
)
# Début des checkers cohérents
before = df.shape[0]
df = df[df["scheduled_trains"] > 0]
print(f"✅ after 'scheduled_trains > 0' : {df.shape[0]} lignes")
df = df[
    df["cancelled_trains"].ge(0) & df["cancelled_trains"].le(df["scheduled_trains"])
]
print(f"✅ after 'cancelled_trains >= 0 et <= scheduled_trains' : {df.shape[0]} lignes")
df = df[
    df["trains_delayed_dep"].ge(0) & df["trains_delayed_dep"].le(df["scheduled_trains"])
]
print(
    f"✅ after 'trains_delayed_dep >= 0 et <= scheduled_trains' : {df.shape[0]} lignes"
)
df = df[
    df["trains_delayed_arr"].ge(0) & df["trains_delayed_arr"].le(df["scheduled_trains"])
]
print(
    f"✅ after 'trains_delayed_arr >= 0 et <= scheduled_trains' : {df.shape[0]} lignes"
)
# Les valeurs > 15/30/60 doivent être ≥ 0 ou 0 si null/négatif
for col in ["trains_delayed_15min", "trains_delayed_30min", "trains_delayed_60min"]:
    df[col] = df[col].apply(lambda x: 0 if pd.isnull(x) or x < 0 else x)
    print(f"✅ after remplacement '{col} < 0 ou null' par 0 : {df.shape[0]} lignes")

# Vérification que les pourcentages des raisons de retard sont >= 0 ou 0 si null/négatif
delay_reason_cols = [
    "pct_delay_external",
    "pct_delay_infrastructure",
    "pct_delay_traffic_mgmt",
    "pct_delay_rolling_stock",
    "pct_delay_station_mgmt",
    "pct_delay_passenger",
]
for col in delay_reason_cols:
    df[col] = df[col].apply(lambda x: 0 if pd.isnull(x) or x < 0 else x)
    print(f"✅ after remplacement '{col} < 0 ou null' par 0 : {df.shape[0]} lignes")

after = df.shape[0]
print(f"✅ Données nettoyées et cohérentes : {after} lignes (sur {before})")

## 🏷️ Validation des Gares avec la Liste Officielle

In [ ]:
gares_valides = set(gares["LIBELLE"].unique())
villes_valides = set(cities["city_lower"].unique())
before_filter = df.shape[0]


def best_gare_match_same_len(name, gares_valides):
    if pd.isna(name) or not isinstance(name, str) or name.strip() == "":
        return np.nan
    if name in gares_valides:
        return name
    # Cherche la gare de même longueur la plus proche
    same_len_gares = [
        g for g in gares_valides if isinstance(g, str) and len(g) == len(name)
    ]
    if not same_len_gares:
        return np.nan
    # Correction : retourne le nom corrigé (le plus proche)
    best_match = min(
        same_len_gares, key=lambda g: distance.Levenshtein.distance(name, g)
    )
    return best_match


def clean_station(name, gares_valides, villes_valides):
    if pd.isna(name) or not isinstance(name, str) or name.strip() == "":
        return np.nan
    if name in gares_valides:
        return name
    if name in villes_valides:
        return name
    gare_corr = best_gare_match_same_len(name, gares_valides)
    if isinstance(gare_corr, str):
        return gare_corr

    return np.nan


df["departure_station"] = df["departure_station"].apply(
    lambda x: clean_station(x, gares_valides, villes_valides)
)
df["arrival_station"] = df["arrival_station"].apply(
    lambda x: clean_station(x, gares_valides, villes_valides)
)

df = df[df["departure_station"].notna() & df["arrival_station"].notna()]

print(
    f"✅ Lignes conservées après correction best match gares (même taille) + villes exactes : {df.shape[0]} (sur {before_filter})"
)

## 🛠️ Enrichissement des Données

In [ ]:
df["month"] = df["date"].dt.month
df["year"] = df["date"].dt.year
df["route"] = df["departure_station"] + " ➜ " + df["arrival_station"]

## 📊 Statistiques Globales

In [ ]:
print(
    "📊 Moyenne globale de retard au départ : {:.2f} min".format(
        df["avg_dep_delay"].mean()
    )
)
print(
    "📊 Moyenne globale de retard à l’arrivée : {:.2f} min".format(
        df["avg_arr_delay"].mean()
    )
)

## 📆 Retards Moyens par Mois

In [ ]:
monthly_delay = df.groupby("month")["avg_dep_delay"].mean()
labels = [
    "Janvier",
    "Février",
    "Mars",
    "Avril",
    "Mai",
    "Juin",
    "Juillet",
    "Août",
    "Septembre",
    "Octobre",
    "Novembre",
    "Décembre",
]

sns.lineplot(x=monthly_delay.index, y=monthly_delay.values, marker="o")
plt.title("Moyenne des retards au départ par mois (2018-2024)")
plt.xlabel("Mois")
plt.ylabel("Retard moyen (min)")
plt.xticks(ticks=range(1, 13), labels=labels, rotation=45)
plt.show()

## 🔥 Carte de Chaleur des Retards (Année x Mois)

In [ ]:
# Juste avant la heatmap :
df["month"] = df["date"].dt.month.astype("Int64")
df["year"] = df["date"].dt.year.astype("Int64")

# Construction de la matrice année x mois complète
heat = (
    df.groupby(["year", "month"])["avg_dep_delay"]
    .mean()
    .unstack(fill_value=np.nan)
    .reindex(columns=range(1, 13))  # force l'ordre des mois de 1 à 12
)

labels = [
    "Janvier",
    "Février",
    "Mars",
    "Avril",
    "Mai",
    "Juin",
    "Juillet",
    "Août",
    "Septembre",
    "Octobre",
    "Novembre",
    "Décembre",
]

sns.heatmap(heat, cmap="coolwarm", annot=True, fmt=".1f")
plt.title("Moyenne des retards par année et mois")
plt.xlabel("Mois")
plt.ylabel("Année")
plt.xticks(ticks=np.arange(12) + 0.5, labels=labels, rotation=45)
plt.show()

## 🚄 Routes avec les Retards Moyens les Plus Longs

In [ ]:
top_routes = (
    df.groupby("route")["avg_dep_delay"].mean().sort_values(ascending=False).head(10)
)

sns.barplot(
    x=top_routes.values,
    y=top_routes.index,
    hue=top_routes.values,
    palette="dark:salmon_r",
    dodge=False,
    legend=False,
)
plt.title("Routes avec les retards moyens les plus longs (De 2018 à 2024)")
plt.xlabel("Retard moyen (min)")
plt.ylabel("Route")
plt.show()

## 📍 Gares avec le plus d’enregistrements de retards

In [ ]:
# First visualization: stations with most delay records
gares_plus_enreg = (
    df.groupby("departure_station").size().sort_values(ascending=False).head(10)
)

sns.barplot(
    x=gares_plus_enreg.values,
    y=gares_plus_enreg.index,
    hue=gares_plus_enreg.index,
    palette="Reds_r",
    dodge=False,
    legend=False,
)

plt.title("Gares avec le plus d'enregistrements de retards (2018-2024)")
plt.xlabel("Nombre d'enregistrements")
plt.ylabel("Gare de départ")
plt.tight_layout()
plt.show()

df["trains_arrives"] = df["scheduled_trains"] - df["cancelled_trains"]
df["trains_arrives"] = df["trains_arrives"].clip(lower=1)

delay_sum = df.groupby("departure_station")["avg_arr_delay"].sum()
trains_arrived_sum = df.groupby("departure_station")["trains_arrives"].sum()
retard_par_train = (
    (delay_sum / trains_arrived_sum).sort_values(ascending=False).head(10)
)

sns.barplot(
    x=retard_par_train.values,
    y=retard_par_train.index,
    hue=retard_par_train.index,
    palette="Blues_r",
    dodge=False,
    legend=False,
)
plt.title("Moyenne de retard par train arrivé en gare entre 2018 et 2024")
plt.xlabel("Retard moyen par train arrivé (min)")
plt.ylabel("Gare de départ")
plt.tight_layout()
plt.show()

## ❌ Gares les moins ponctuelles (retard moyen élevé)

In [ ]:
gares_moins_ponctuelles = (
    df.groupby("departure_station")["avg_dep_delay"]
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

sns.barplot(
    x=gares_moins_ponctuelles.values,
    y=gares_moins_ponctuelles.index,
    hue=gares_moins_ponctuelles.values,
    palette="Oranges",
    dodge=False,
    legend=False,
)
plt.title("Gares les moins ponctuelles (retard moyen élevé) (2018-2024)")
plt.xlabel("Retard moyen (min)")
plt.ylabel("Gare de départ")
plt.show()

## ✅ Gares les plus ponctuelles (retard moyen faible)

In [ ]:
gares_plus_ponctuelles = (
    df.groupby("departure_station")["avg_dep_delay"].mean().sort_values().head(10)
)

sns.barplot(
    x=gares_plus_ponctuelles.values,
    y=gares_plus_ponctuelles.index,
    hue=gares_plus_ponctuelles.values,
    palette="Greens",
    dodge=False,
    legend=False,
)
plt.title("Gares les plus ponctuelles (retard moyen faible) de 2018 à 2024")
plt.xlabel("Retard moyen (min)")
plt.ylabel("Gare de départ")
plt.show()

## 💾 Export des Données Nettoyées

In [ ]:
df.to_csv("cleaned_dataset.csv", index=False, sep=";")
print("💾 Données nettoyées exportées dans clean_dataset.csv")